## Shrink build_tables_output

In [1]:
import pandas as pd

In [2]:
datadir = '/home/freddy/insight/data/'
filename = datadir + 'build_tables_output.csv'

In [3]:
df = pd.read_csv(filename)
nrows0 = df.shape[0]

/home/freddy/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df=df[df.terrain != 'Multi']

In [5]:
df=df.drop(columns=['Unnamed: 0'], axis=1)

In [6]:
cols = ['age_group','name','sex', 'race_title', 
        'terrain', 'race_date', 'meeting_id']

In [7]:
for i in cols:
    df = df[pd.notnull(df[i])]

In [8]:
df['chip_time'].fillna(0, inplace=True)
df['gun_time'].fillna(0, inplace=True)

In [9]:
nrows1 = df.shape[0]
frac = (1-nrows1/nrows0)*100
print('percent lost = %1.3f%%' % frac)

percent lost = 27.459%


In [10]:
IDs = df['meeting_id'].tolist()

In [11]:
def cut_up_url(url):
    flag = 'meetingid='
    start = url.find(flag)+len(flag) 
    end   = url.find('&')
    substr = url[start:end]
    return int(substr)

In [12]:
def handle_gpx(filename):
    gpx = pd.read_csv(filename, header=0)
    gpx['meeting_id'] = gpx['URL'].apply(lambda x: cut_up_url(x))
    #gpx.rename(index=str, columns={'race_data':'race_date'})
    return gpx

In [13]:
gpx = handle_gpx(datadir + 'gpx_discover.csv')
gpx_IDs = gpx['meeting_id'].tolist()

In [14]:
# Make everything unique, and then find the overlap
uniqueIDs = set(IDs)
gpx_uniqueIDs = set(gpx_IDs)

In [15]:
overlapIDs = list(uniqueIDs & gpx_uniqueIDs)

In [16]:
dfnew = df[df['meeting_id'].isin(overlapIDs)]

In [17]:
#df.groupby(['name'], as_index=False).size().sort_values(ascending=False)
#df.groupby(['race_location'], as_index=False).size().sort_values(ascending=False)

In [18]:
dfnew.head()

,age_group,chip_time,gun_time,name,position,race_title,sex,terrain,race_date,race_location,meeting_id,uniqueID
1134964,U23,25:15,25:15,Callum Charleston,1,5M,M,Road,26 Dec 18,Witham,257748.0,3874.0
1134965,U23,25:27,25:28,Ramadan Osman,2,5M,M,Road,26 Dec 18,Witham,257748.0,3874.0
1134966,SEN,25:33,25:35,David Green,3,5M,M,Road,26 Dec 18,Witham,257748.0,3874.0
1134967,SEN,25:44,25:45,Stephen Strange,4,5M,M,Road,26 Dec 18,Witham,257748.0,3874.0
1134968,V40,25:54,25:55,Crispian Bloomfield,5,5M,M,Road,26 Dec 18,Witham,257748.0,3874.0


In [19]:
#dfnew.to_csv('data_overlaps_with_gpx.csv',sep=',')

In [23]:
dfnew.loc[dfnew.race_title=='10k','race_title'] = '10K'
dfnew=dfnew[dfnew.race_title == '10K']

In [25]:
#dfnew.groupby(['race_location','race_title'], as_index=False).size().sort_values(ascending=False)